Компанией был куплен крупный сервис для чтения книг по подписке.

**Задача:** проанализировать базу данных.

**Таблица books** - содержит данные о книгах.

|столбец|значение|
|:------|:-------|
|book_id|идентификатор книги|
|author_id|идентификатор автора|
|title|название книги|
|num_pages|количество страниц|
|publication_date|дата публикации книги|
|publisher_id|идентификатор издателя|


**Таблица authors** - cодержит данные об авторах.

|столбец|значение|
|:------|:-------|
|author_id|идентификатор автора|
|author|имя автора|


**Таблица publishers** - cодержит данные об издательствах.

|столбец|значение|
|:------|:-------|
|publisher_id|идентификатор издательства|
|publisher|название издательства|


**Таблица ratings** -cодержит данные о пользовательских оценках книг.

|столбец|значение|
|:------|:-------|
|rating_id|идентификатор оценки|
|book_id|идентификатор книги|
|username|имя пользователя, оставившего оценку|
|rating|оценка книги|

**Таблица reviews** - cодержит данные о пользовательских обзорах на книги.

|столбец|значение|
|:------|:-------|
|review_id|идентификатор обзора|
|book_id|идентификатор книги|
|username|имя пользователя, написавшего обзор|
|text|текст обзора|

In [1]:
import pandas as pd
import warnings; warnings.filterwarnings(action='once')
import sqlalchemy as sa

In [2]:
from sqlalchemy import create_engine
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

In [3]:
query = '''

SELECT COUNT(*)
FROM books

'''

pd.io.sql.read_sql(query, con = engine)

,count
0,1000


В таблице books у нас 1000 строк.

In [4]:
query = '''

SELECT *
FROM books
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''

SELECT COUNT(*)
FROM authors
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,count
0,636


В таблице authors 636 строк.

In [6]:
query = '''

SELECT *
FROM authors
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
query = '''

SELECT COUNT(*)
FROM publishers

'''

pd.io.sql.read_sql(query, con = engine)

,count
0,340


В таблице publishers 340 строк.

In [8]:
query = '''

SELECT *
FROM publishers
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
query = '''

SELECT COUNT(*)
FROM ratings

'''

pd.io.sql.read_sql(query, con = engine)

,count
0,6456


В таблице ratings 6456 строк.

In [10]:
query = '''

SELECT *
FROM ratings
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
query = '''

SELECT COUNT(*)
FROM reviews

'''

pd.io.sql.read_sql(query, con = engine)

,count
0,2793


В столбце reviews 2793 строки.

In [12]:
query = '''

SELECT *
FROM reviews
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Посчитаем, сколько книг вышло после 1 января 2000 года

In [13]:
query = '''

SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'

'''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [14]:
query = '''

SELECT b.title, COUNT(DISTINCT rev.review_id) AS count_rev, ROUND(AVG(r.rating),1) AS avg_rating
FROM books AS b

LEFT JOIN reviews AS rev ON b.book_id = rev.book_id

LEFT JOIN  ratings AS r ON b.book_id = r.book_id

GROUP BY b.book_id, b.title
ORDER BY count_rev DESC

'''

pd.io.sql.read_sql(query, con = engine)

,title,count_rev,avg_rating
0,Twilight (Twilight #1),7,3.7
1,Water for Elephants,6,4.0
2,The Glass Castle,6,4.2
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.4
4,The Curious Incident of the Dog in the Night-Time,6,4.1
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.7
996,The Natural Way to Draw,0,3.0
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.0
998,Essential Tales and Poems,0,4.0


Самое большое число обзоров на книгу равно 7. Зависимости между числом обзоров и рейтингом не наблюдается.

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [15]:
query = ''' 

SELECT publisher,
COUNT (book_id) OVER (PARTITION BY publisher_id) AS book_count
FROM (SELECT b.publisher_id, publisher, book_id, num_pages
FROM books AS b LEFT JOIN publishers AS pub ON pub.publisher_id = b.publisher_id) AS pub_t

WHERE num_pages > 50
ORDER BY 2 DESC
LIMIT 1

'''
pd.io.sql.read_sql(query, con = engine)

,publisher,book_count
0,Penguin Books,42


### Определим автора с самой высокой средней оценкой книг — будем учитывать только книги с 50 и более оценками

In [16]:
query = '''

WITH 
tab AS (SELECT book_id
FROM ratings
GROUP BY book_id
HAVING COUNT(rating_id) > 50)

SELECT  a.author, ROUND(AVG(r.rating),1) AS avg_rating
FROM books AS b

INNER JOIN tab ON b.book_id = tab.book_id

INNER JOIN authors AS a ON b.author_id = a.author_id

INNER JOIN ratings AS r ON tab.book_id = r.book_id

GROUP BY a.author_id, a.author
ORDER BY AVG(r.rating) DESC

LIMIT 1

'''

pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.3


### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [17]:
query = '''

SELECT COUNT(DISTINCT rev.text)/ COUNT(DISTINCT rev.username) AS avg_rev_count
FROM ratings AS r

INNER JOIN reviews AS rev ON r.username = rev.username

WHERE r.username in (SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 48)

'''

pd.io.sql.read_sql(query, con = engine)

,avg_rev_count
0,24


**Выводы:**

После исследования, можно проанализировать базу данных:
- после 1 января 2000 года вышло 819 книг;
- самое большое число обзоров на книгу равно 7. Зависимости между числом обзоров и рейтингом не наблюдается;
- исключив из анализа брошюры, определено издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books (42 книги);
- автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré (4,3);
- среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.